In [4]:
"""
NEURON simulation of a one-dimensional network of excitatory and inhibitory neurons
with spike-timing-dependent plasticity (STDP) to study self-organization
under different noise conditions.

Network Structure:
- 100 Excitatory (E) neurons: Two compartments (soma, dendrite).
- 25 Inhibitory (I) neurons: Single compartment (SST-like).
- 100 Poisson input sources projecting to E-neuron dendrites.

Connections:
1.  Input -> E (dendrite): Spatially tuned, Bell-shaped weights. Fixed.
2.  E -> I (soma): Excitatory, plastic with Hebbian STDP.
3.  I -> E (distal dendrite): Inhibitory feedback. Fixed.
4.  Noise 1 -> E (soma): Poisson noise source.
5.  Noise 2 -> E (distal dendrite): Poisson noise source.

The script sets up the network, runs the simulation, and records spike data.
"""

import random
from neuron import h, gui
import numpy as np

In [12]:
# Make sure to have NEURON installed: pip install neuron

# --- 1. Simulation Parameters ---
h.load_file('stdrun.hoc')
SIM_DURATION = 500  # ms
h.tstop = SIM_DURATION
h.dt = 0.1 # ms
random.seed(42)
np.random.seed(42)

# --- 2. Network Parameters ---
N_E = 100  # Number of excitatory neurons
N_I = 25   # Number of inhibitory neurons
N_INPUT = 100 # Number of external input sources
SPACE_EXTENT = 100  # um, extent of the 1D space

# --- 3. Neuron and Synapse Parameters ---
# Leaky Integrate-and-Fire (LIF) parameters
LIF_TAU = 20.0    # Membrane time constant (ms)
LIF_V_REST = -65.0 # Resting potential (mV)
LIF_V_RESET = -65.0 # Reset potential (mV)
LIF_V_THRESH = -50.0 # Firing threshold (mV)
LIF_R = 10.0 # Membrane resistance (Mohm)

# Synaptic parameters
E_SYN_TAU1 = 0.2  # Excitatory synapse rise time (ms)
E_SYN_TAU2 = 3.0   # Excitatory synapse decay time (ms)
I_SYN_TAU1 = 0.5  # Inhibitory synapse rise time (ms)
I_SYN_TAU2 = 10.0  # Inhibitory synapse decay time (ms)

# STDP parameters (for E -> I connections)
STDP_TAU_PLUS = 20  # ms
STDP_TAU_MINUS = 20 # ms
STDP_A_PLUS = 0.01  # LTP strength
STDP_A_MINUS = 0.012 # LTD strength
W_MAX = 0.01        # Maximum synaptic weight

In [ ]:
def __init__(self, gid)
    self.soma = n.Section(name="soma", cell=self)
    self.dend = n.Section(name="dend", cell=self)
    

In [13]:
# --- 4. Neuron Class Definitions ---

class ENeuron:
    """Two-compartment Excitatory Neuron (soma + dendrite)."""
    def __init__(self, x_pos):
        self.x = x_pos
        self.soma = h.Section(name='soma', cell=self)
        self.dend = h.Section(name='dend', cell=self)

        # Geometry
        self.soma.L = self.soma.diam = 20 # um
        self.dend.L = 500 # um
        self.dend.diam = 2 # um
        self.dend.nseg = 3 # 3 segments for distal/proximal targeting

        # Connection
        self.dend.connect(self.soma(1))

        # Biophysics (LIF properties)
        for sec in [self.soma, self.dend]:
            sec.Ra = 100 # Axial resistance (Ohm*cm)
            sec.cm = 1   # Membrane capacitance (uF/cm^2)
            # Add LIF channel
            sec.insert(h.lif)
            lif_chan = sec.lif
            lif_chan.tau = LIF_TAU
            lif_chan.v_rest = LIF_V_REST
            lif_chan.v_thresh = LIF_V_THRESH
            lif_chan.v_reset = LIF_V_RESET
            lif_chan.R = LIF_R

        # Spike detector
        self.spike_detector = h.NetCon(self.soma(0.5)._ref_v, None, sec=self.soma)
        self.spike_times = h.Vector()
        self.spike_detector.record(self.spike_times)

In [9]:
class INeuron:
    """Single-compartment Inhibitory Neuron."""
    def __init__(self, x_pos):
        self.x = x_pos
        self.soma = h.Section(name='soma', cell=self)
        self.soma.L = self.soma.diam = 15 # um

        # Biophysics (LIF properties)
        self.soma.Ra = 100
        self.soma.cm = 1
        self.soma.insert('lif')
        lif_chan = self.soma.lif
        lif_chan.tau = LIF_TAU
        lif_chan.v_rest = LIF_V_REST
        lif_chan.v_thresh = LIF_V_THRESH
        lif_chan.v_reset = LIF_V_RESET
        lif_chan.R = LIF_R

        # Spike detector
        self.spike_detector = h.NetCon(self.soma(0.5)._ref_v, None, sec=self.soma)
        self.spike_times = h.Vector()
        self.spike_detector.record(self.spike_times)

In [14]:
# --- 5. Network Creation ---
print("Creating neuron populations...")
e_neurons = [ENeuron(x) for x in np.linspace(0, SPACE_EXTENT, N_E)]
i_neurons = [INeuron(x) for x in np.linspace(0, SPACE_EXTENT, N_I)]

Creating neuron populations...


AttributeError: 'hoc.HocObject' object has no attribute 'lif'

In [ ]:
# --- 6. Input and Noise Sources ---
print("Setting up input and noise sources...")
# Poisson-distributed spiking inputs
input_sources = []
input_spikes = []
for i in range(N_INPUT):
    source = h.NetStim()
    source.interval = 20 # Average interval 20 ms -> 50 Hz
    source.number = 1e9 # Effectively infinite spikes
    source.noise = 1 # Poisson process
    source.start = 0
    input_sources.append(source)
    # Record spike times for potential analysis
    spike_vec = h.Vector()
    nc = h.NetCon(source, None)
    nc.record(spike_vec)
    input_spikes.append(spike_vec)

# Noise sources
noise_soma = h.NetStim()
noise_soma.interval = 10 # 100 Hz noise
noise_soma.number = 1e9
noise_soma.noise = 1
noise_soma.start = 0

noise_dend = h.NetStim()
noise_dend.interval = 10 # 100 Hz noise
noise_dend.number = 1e9
noise_dend.noise = 1
noise_dend.start = 0

In [ ]:
# --- 7. Synaptic Connections ---
print("Building synaptic connections...")
connections = {'input_e': [], 'e_i': [], 'i_e': [], 'noise_soma': [], 'noise_dend': []}

# a) Input -> E neurons (spatially tuned)
tuning_width = 10 # um, width of the Bell-shaped tuning curve
max_weight_input = 0.02
for i, src in enumerate(input_sources):
    # Each input source is centered on one E neuron's location
    center_x = e_neurons[i].x
    for j, e_neuron in enumerate(e_neurons):
        dist = abs(e_neuron.x - center_x)
        weight = max_weight_input * np.exp(-(dist**2) / (2 * tuning_width**2))

        if weight > 1e-4: # Connect only if weight is significant
            syn = h.Exp2Syn(e_neuron.dend(0.5)) # Target proximal dendrite
            syn.tau1 = E_SYN_TAU1
            syn.tau2 = E_SYN_TAU2
            nc = h.NetCon(src, syn)
            nc.delay = 1 # ms
            nc.weight[0] = weight
            connections['input_e'].append(nc)

# b) E -> I neurons (excitatory, plastic)
# Randomly connect 10% of E neurons to each I neuron
conn_prob_ei = 0.1
for i_neuron in i_neurons:
    for e_neuron in e_neurons:
        if random.random() < conn_prob_ei:
            syn = h.Exp2Syn(i_neuron.soma(0.5))
            syn.tau1 = E_SYN_TAU1
            syn.tau2 = E_SYN_TAU2
            
            # Create NetCon for STDP
            nc = h.NetCon(e_neuron.soma(0.5)._ref_v, syn, sec=e_neuron.soma)
            nc.delay = 1 + random.uniform(0, 1)
            nc.weight[0] = random.uniform(0.001, 0.005) # Initial random weight

            # Add STDP mechanism
            stdp = h.STDP(syn)
            stdp.tau_plus = STDP_TAU_PLUS
            stdp.tau_minus = STDP_TAU_MINUS
            stdp.a_plus = STDP_A_PLUS
            stdp.a_minus = STDP_A_MINUS
            stdp.w_max = W_MAX
            
            connections['e_i'].append({'netcon': nc, 'stdp': stdp})


# c) I -> E neurons (inhibitory feedback)
conn_prob_ie = 0.4 # Each I neuron inhibits 40% of E neurons
max_weight_inhib = 0.005
for i_neuron in i_neurons:
    for e_neuron in e_neurons:
        if random.random() < conn_prob_ie:
            # Inhibitory synapses target distal dendrites
            syn = h.Exp2Syn(e_neuron.dend(1))
            syn.e = -80 # Reversal potential for inhibition
            syn.tau1 = I_SYN_TAU1
            syn.tau2 = I_SYN_TAU2
            nc = h.NetCon(i_neuron.soma(0.5)._ref_v, syn, sec=i_neuron.soma)
            nc.delay = 1 + random.uniform(0, 1)
            nc.weight[0] = max_weight_inhib
            connections['i_e'].append(nc)

# d) Noise -> E neurons
noise_weight = 0.01
for e_neuron in e_neurons:
    # Noise to soma
    syn_soma = h.Exp2Syn(e_neuron.soma(0.5))
    syn_soma.tau1 = E_SYN_TAU1
    syn_soma.tau2 = E_SYN_TAU2
    nc_soma = h.NetCon(noise_soma, syn_soma)
    nc_soma.delay = 1
    nc_soma.weight[0] = noise_weight
    connections['noise_soma'].append(nc_soma)

    # Noise to distal dendrite
    syn_dend = h.Exp2Syn(e_neuron.dend(1))
    syn_dend.tau1 = E_SYN_TAU1
    syn_dend.tau2 = E_SYN_TAU2
    nc_dend = h.NetCon(noise_dend, syn_dend)
    nc_dend.delay = 1
    nc_dend.weight[0] = noise_weight
    connections['noise_dend'].append(nc_dend)

print(f"Total connections: Input->E: {len(connections['input_e'])}, "
      f"E->I: {len(connections['e_i'])}, I->E: {len(connections['i_e'])}")


# --- 8. Simulation and Recording ---
# Record voltages for one E neuron
v_soma_rec = h.Vector().record(e_neurons[50].soma(0.5)._ref_v)
v_dend_rec = h.Vector().record(e_neurons[50].dend(0.5)._ref_v)
t_rec = h.Vector().record(h._ref_t)

print("Starting simulation...")
h.init()
h.run(SIM_DURATION)
print("Simulation finished.")

# --- 9. Data Extraction and Analysis ---
# Example of how to access the data.
# For a full analysis, you would typically save this data to files
# and use libraries like Matplotlib for plotting.

print("\n--- Simulation Results ---")
total_e_spikes = sum(len(n.spike_times) for n in e_neurons)
total_i_spikes = sum(len(n.spike_times) for n in i_neurons)
print(f"Total E spikes: {total_e_spikes} (Avg rate: {total_e_spikes / N_E / (SIM_DURATION/1000):.2f} Hz)")
print(f"Total I spikes: {total_i_spikes} (Avg rate: {total_i_spikes / N_I / (SIM_DURATION/1000):.2f} Hz)")

# Show final weights for plastic synapses
final_weights_ei = [c['netcon'].weight[0] for c in connections['e_i']]
print(f"E->I final weights: Min={min(final_weights_ei):.5f}, Max={max(final_weights_ei):.5f}, Avg={np.mean(final_weights_ei):.5f}")

# You can create raster plots using the recorded spike times.
# Example for E neurons:
# for i, neuron in enumerate(e_neurons):
#     plt.plot(list(neuron.spike_times), [i] * len(neuron.spike_times), 'k.')
# plt.show()